# Tech Intro

## Main tables

In [102]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [103]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[1] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [104]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1119 14:12:21.381275 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1119 14:12:21.381664 40276 sched.cpp:241] Version: 1.11.0
I1119 14:12:21.382014 10561 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1119 14:12:21.382092 10561 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1119 14:12:21.383656 10568 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75337


## Read the spec-file from Hadoop and show some data sample

In [105]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+--------+------------+-----------+-----------------------+----------+--------+-----+-----+
|lead_id |utm_campaign|utm_content|utm_source             |utm_medium|utm_term|gclid|yclid|
+--------+------------+-----------+-----------------------+----------+--------+-----+-----+
|21715024|null        |null       |ufa.perspektiva24.com  |null      |null    |null |null |
|21715204|null        |null       |google                 |null      |null    |null |null |
|21715420|null        |null       |kazan.perspektiva24.com|null      |null    |null |null |
|21715549|null        |null       |ufa.perspektiva24.com  |null      |null    |null |null |
|21715555|null        |null       |perspektiva24.com      |null      |null    |null |null |
|21716637|null        |null       |ufa.perspektiva24.com  |null      |null    |null |null |
|21716732|null        |null       |ufa.perspektiva24.com  |null      |null    |null |null |
|21717011|null        |null       |google                 |null      |null    |n

## Show columns' properties

In [106]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- lead_id: string (nullable = true)
 |-- utm_campaign: string (nullable = true)
 |-- utm_content: string (nullable = true)
 |-- utm_source: string (nullable = true)
 |-- utm_medium: string (nullable = true)
 |-- utm_term: string (nullable = true)
 |-- gclid: string (nullable = true)
 |-- yclid: string (nullable = true)



## Total rows (count)

In [108]:
eda_df.count()

1964

## Simple func to execute and show the result of the SQL-instruction

In [109]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [110]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [111]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|1964    |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [112]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid
0,0,1963,1951,1174,1963,1963,1964,1125


In [113]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid
0,0,0,0,0,0,0,0,0


In [114]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid
0,0,0,0,0,0,0,0,42


In [115]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
lead_id,0,0,0
utm_campaign,1963,0,0
utm_content,1951,0,0
utm_source,1174,0,0
utm_medium,1963,0,0
utm_term,1963,0,0
gclid,1964,0,0
yclid,1125,0,42


In [116]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [117]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
lead_id,0.000,0.0,0.000
utm_campaign,99.949,0.0,0.000
utm_content,99.338,0.0,0.000
utm_source,59.776,0.0,0.000
utm_medium,99.949,0.0,0.000
utm_term,99.949,0.0,0.000
gclid,100.000,0.0,0.000
yclid,57.281,0.0,2.138


### SQL-instructions

In [118]:
#Find dupes or misunderstanding or useless static values 981 633 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+-----------------------+
|count(DISTINCT lead_id)|
+-----------------------+
|1964                   |
+-----------------------+

+----------------------------+
|count(DISTINCT utm_campaign)|
+----------------------------+
|1                           |
+----------------------------+

+---------------------------+
|count(DISTINCT utm_content)|
+---------------------------+
|2                          |
+---------------------------+

+--------------------------+
|count(DISTINCT utm_source)|
+--------------------------+
|35                        |
+--------------------------+

+--------------------------+
|count(DISTINCT utm_medium)|
+--------------------------+
|1                         |
+--------------------------+

+------------------------+
|count(DISTINCT utm_term)|
+------------------------+
|1                       |
+------------------------+

+---------------------+
|count(DISTINCT gclid)|
+---------------------+
|0                    |
+---------------------+

+------------

In [129]:
#Find dupes or misunderstanding or useless static values 1964 total

execute("SELECT DISTINCT(utm_source)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY utm_source DESC")

execute("SELECT DISTINCT(utm_content)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY utm_content ASC")

execute("SELECT DISTINCT(yclid)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY yclid DESC")


+-----------------------------+
|utm_source                   |
+-----------------------------+
|yandex                       |
|yalutorovsk.perspektiva24.com|
|vn.perspektiva24.com         |
|ufa.perspektiva24.com        |
|ufa-sever.perspektiva24.com  |
|test                         |
|taplink.cc                   |
|string                       |
|spb.perspektiva24.com        |
|samoletplus.ru               |
|samolet.ru                   |
|ru.m.wikipedia.org           |
|podolsk.perspektiva24.com    |
|perspektiva24.com            |
|perspektiva-24.ru            |
|oferta.onperspektiva24.com   |
|mscrm.mtt.ru                 |
|m.vk.com                     |
|link.2gis.ru                 |
|link.2gis.com                |
+-----------------------------+
only showing top 20 rows

+-----------+
|utm_content|
+-----------+
|null       |
|string     |
|{ad_id}    |
+-----------+

+--------------------+
|yclid               |
+--------------------+
|string              |
|57665986768452

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [101]:
spark.stop()

I1119 14:12:15.289315 10390 sched.cpp:2174] Asked to stop the driver
I1119 14:12:15.289451 10556 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75308
